<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/JonasNylund/Lab1_jonasnylund_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract
In this lab, we construct a few algorithms for basic matrix operations, and compare their results with numpy for correctness.


In [0]:
"""This program is a lab report in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Jonas Nylund (jonasnyl@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.


# Introduction
Systems of equations can be seen as matrix equations. Solving these systems of equations are analogous to solving a matrix equation. Linear algebra is the core of many numerical methods in computer science, as any real world function can be approximated as linear in small enough intervalls. Therefore it is valuable to know and be albe to solve linear algebra equations. This is why we in this lab implement a few matrix and vector functions.


# Methods

We define a class for matrices, with a add and multiply function. The matrix can be any size.

We then define a subclass, Vector, that is a special case of matrices where one dimension is 1. A vector still has a 'direction', a 1xn vector is a row vector, and a nx1 vector is a column vector. One can be transposed into the other.

For vectors, we define a function scalarMultiply, that computed the dot product of two vectors. For this to be defined, they need to have the same size.

In matrix multiplication, each cell in the result matrix is computed as the dot product of the corresponding row- and column-vectors in the input matrices. We therefore use the vector class to compute each cell value in the output matrix.

We also calculate the euclidian norm of a vector, which is the length of the vector. Having the functionanlty, we can also calculate the distance between two vectors, by subtracting one from the other and taking the norm. 

In [0]:
import numpy as np


class Matrix:

  def __init__(self, data: np.ndarray):
    if(type(data) != np.ndarray):
      raise TypeError("Illegal argument type, should be np.Array");

    self.data = data;
    self.size = data.shape;


  def __str__(self):
    return str(self.data);


  def __eq__(self, other):
    if(type(other) != Matrix):
      return False;
    return np.array_equal(self.data, other.data);


  def __ne__(self, other):
    return not self.__eq__(other);


  def negate(self):
    result = np.zeros(self.data.shape);
    for i in range(self.data.shape[0]):
      for j in range(self.data.shape[1]):
        result[i][j] = -self.data[i][j];
    return Matrix(result);


  def add(self, matrix: 'Matrix'):
    if(self.data.shape != matrix.data.shape):
      raise ValueError("Shape error, matrices must share dimension");

    result = np.zeros(self.data.shape);
    for i in range(self.data.shape[0]):
      for j in range(self.data.shape[1]):
        result[i][j] = self.data[i][j] + matrix.data[i][j];
    return Matrix(result);


  def subtract(self, matrix: 'Matrix'):
    temp = matrix.negate();
    return self.add(temp);


  def multiply(self, matrix: 'Matrix'):
    if(self.data.shape[1] != matrix.data.shape[0]):
      raise ValueError("Shape error, matrices must share dimension");

    result = np.zeros((self.data.shape[0], matrix.data.shape[1]));
    for row in range(self.data.shape[0]):
      for col in range(matrix.data.shape[1]):
        result[row][col] = self.getRow(row).scalarMultiply(matrix.getCol(col).transpose());

    return Matrix(result);


  def getRow(self, index: int):
    if(index >= self.data.shape[0]):
      raise IndexError("Index out of bounds");

    return Vector(self.data[index,:]);


  def getCol(self, index: int):
    if(index >= self.data.shape[1]):
      raise IndexError("Index out of bounds");

    return Vector(self.data[:,index]).transpose();


  def transpose(self):
    return Vector(self.data.transpose());


  def print(self):
    print(self.data);




class Vector(Matrix):

  def __init__(self,data: np.ndarray):
    if(type(data) != np.ndarray):
      raise TypeError("Illegal argument type, should be np.Array");
    if(data.ndim > 1 and not (data.shape[0] == 1 or data.shape[1] == 1)):
      raise TypeError("Input is not a vector");

    if(data.ndim == 1):
      data = np.array([data]);

    super(Vector, self).__init__(data);


  def scalarMultiply(self, array: 'Vector'):
    if(self.data.shape != array.data.shape):
      raise ValueError("Illegal argument. Arrays must be of the same size and shape");

    result = 0;
    for i in range(len(self.data)):
      for j in range(len(self.data[i])):
        result += self.data[i][j]*array.data[i][j];
    return result;

  def add(self, other):
    return Vector(super(Vector, self).add(other).data);

  def norm(self):
    val = 0;
    for i in range(self.data.shape[0]):
      for j in range(self.data.shape[1]):
        val += self.data[i][j]**2;

    return np.sqrt(val);


  def dist(self, other: 'Vector'):
    return self.subtract(other).norm();




#Result
Testing that the result is correct is done by comparing the output of the function to the output from numpy matrix multiplyer. We test each of the functions with 100 random matrixes and vectors. The tests are performed as unit tests.

In [0]:
def randomVector(len):
  a = np.zeros(len);
  for i in range(len):
    a[i] = random.random()*200-100;

  return a;


def randomMatrix(n, m):
  mat = np.zeros((n,m));

  for i in range(n):
    for j in range(m):
      mat[i][j] = random.random()*200-100;

  return mat;



In [6]:
import unittest, random

class TestMatrixMethods(unittest.TestCase):
  

  def test_scalarProduct(self):
    for i in range(100):
      l = random.randint(2,10);
      v1 = randomVector(l);
      v2 = randomVector(l);
      self.assertEqual(np.dot(v1,v2), Vector(v1).scalarMultiply(Vector(v2)));

    with self.assertRaises(ValueError):
      v1 = randomVector(4);
      v2 = randomVector(5);
      Vector(v1).scalarMultiply(Vector(v2));

    with self.assertRaises(TypeError):
      Vector(np.array([[1,2,3,4],[1,12,34,56]]));


  def test_matrixVectorMultiplication(self):
    for i in range(100):
      l = random.randint(2,10);
      n = random.randint(2,10);
      x = Vector(randomVector(l)).transpose();
      A = Matrix(randomMatrix(n,l));
      
      self.assertTrue(np.allclose(np.matmul(A.data,x.data), A.multiply(x).data));

    for i in range(100):
      l = random.randint(2,10);
      n = random.randint(2,10);
      x = Vector(randomVector(l));
      A = Matrix(randomMatrix(l, n));
      
      self.assertTrue(np.allclose(np.matmul(x.data,A.data), x.multiply(A).data));


  def test_matrixMatrixMultiplication(self):
    for i in range(100):
      l = random.randint(2,10);
      m = random.randint(2,10);
      n = random.randint(2,10);
      A = Matrix(randomMatrix(l,n));
      B = Matrix(randomMatrix(n,m));

      self.assertTrue(np.allclose(np.matmul(A.data,B.data), A.multiply(B).data));

  def test_norm(self):
    for i in range(100):
      l = random.randint(2,10);
      v1 = Vector(randomVector(l));
      self.assertTrue(np.isclose(v1.norm(), np.linalg.norm(v1.data)));


  def test_dist(self):
    for i in range(100):
      l = random.randint(2,10);
      v1 = Vector(randomVector(l));
      v2 = Vector(randomVector(l));
      self.assertTrue(np.isclose(v1.dist(v2), np.linalg.norm(v1.data-v2.data)));
  
  
  def test_dimensionsErrorCheck(self):
    A = Matrix(randomMatrix(5,4));
    B = Matrix(randomMatrix(5,4));
    x = Vector(randomVector(3));

    with self.assertRaises(ValueError):
      A.multiply(B);

    with self.assertRaises(ValueError):
      A.multiply(x);
      
    with self.assertRaises(ValueError):
      x.multiply(B);


unittest.main(argv=[''], verbosity=2, exit=False)


test_dimensionsErrorCheck (__main__.TestMatrixMethods) ... ok
test_dist (__main__.TestMatrixMethods) ... ok
test_matrixMatrixMultiplication (__main__.TestMatrixMethods) ... ok
test_matrixVectorMultiplication (__main__.TestMatrixMethods) ... ok
test_norm (__main__.TestMatrixMethods) ... ok
test_scalarProduct (__main__.TestMatrixMethods) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.155s

OK


All of the tests succed, so we assume the functions are correct.
# Discussion
These functions use the naive method of multiplication, they might not be very efficient with very large matrices.

Overall, the code seems to work as intended